In [1]:
# [CELL-1] importing the json file in which the scraped data from both the parent links and their corresponding sub-links (up to a depth of 5 levels)

import json
import pandas as pd

file_path= "./nvidia_crawler/output.json"

with open(file_path,"r") as file:
    data= json.load(file)
    
df = pd.DataFrame(data)
print(df.head())

                                                 url  \
0                      https://docs.nvidia.com/cuda/   
1  https://docs.nvidia.com/cuda/cuda-toolkit-rele...   
2  https://docs.nvidia.com/cuda/cuda-features-arc...   
3       https://docs.nvidia.com/cuda/eula/index.html   
4  https://docs.nvidia.com/cuda/cuda-quick-start-...   

                                               title  \
0                    CUDA Toolkit Documentation 12.5   
1                   CUDA 12.5 Update 1 Release Notes   
2                              CUDA Features Archive   
3                                               EULA   
4  1. Introduction — Quick Start Guide 12.5 docum...   

                                             content  
0  <body class="wy-body-for-nav"> \r\n  <div clas...  
1  <body class="wy-body-for-nav"> \r\n  <div clas...  
2  <body class="wy-body-for-nav"> \n  <div class=...  
3  <body class="wy-body-for-nav"> \n  <div class=...  
4  <body class="wy-body-for-nav"> \r\n  <div clas..

In [2]:
#[CELL-2] Changind the content in text format from HTML foramt and then updating

from bs4 import BeautifulSoup
def extract_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.get_text(separator=" ", strip=True)
    return text

# Update the 'content' column with extracted text
df['content'] = df['content'].apply(extract_text)

In [9]:
#[CELL-3] Data Chunking

import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')


def advanced_chunk_text(text, chunk_size=128):
    sentences = text.split('. ')
    if len(sentences) == 1:
        return [text]

    embeddings = model.encode(sentences)
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5).fit(embeddings)
    
    chunks = []
    current_chunk = []
    current_cluster = clustering.labels_[0]

    for sentence, cluster in zip(sentences, clustering.labels_):
        if cluster != current_cluster or len(current_chunk) >= chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_cluster = cluster
        current_chunk.append(sentence)
    
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [5]:
#[CELL-4] importing necessary items from pymilvus

from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [8]:
#[CELL-5] Starting the milvus server

from milvus import default_server

try:
    default_server.start()
    print("Milvus server started successfully!")
except Exception as e:
    print(f"Failed to start Milvus server: {str(e)}")
    exit(1)

Milvus server started successfully!


In [9]:
#[CELL-6] Connecting to the Milvus server using the host and the default port.

from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility, MilvusException
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    connections.connect(alias="default", host='192.168.1.4', port='19530')
    logger.info("Connected to Milvus server.")
except MilvusException as e:
    logger.error(f"Failed to connect to Milvus server: {e}")


INFO:__main__:Connected to Milvus server.


In [10]:
#[CELL-7] Providing name to the collection
collection_name = "new_docs_v2"


In [11]:
#[CELL-8] Defining Schema 
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384)
url_field = FieldSchema(name="url", dtype=DataType.VARCHAR, max_length=512)

schema = CollectionSchema(fields=[id_field, embedding_field, url_field], description="Collection for CUDA documentation")

collection = Collection(name=collection_name, schema=schema)
logger.info(f"Collection '{collection_name}' created with schema: {schema}")



INFO:__main__:Collection 'new_docs_v2' created with schema: {'auto_id': True, 'description': 'Collection for CUDA documentation', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'url', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': False}


In [15]:
#[CELL-9] Vector Database Creation and Inserting in the collection
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def insert_chunks_into_milvus(chunks, urls, collection, model, batch_size=50):
    try:
        entities = []
        for i in tqdm(range(0, len(chunks), batch_size)):
            batch_chunks = chunks[i:i + batch_size]
            batch_urls = urls[i:i + batch_size]

            batch_embeddings = model.encode(batch_chunks)

            for chunk, url, embedding in zip(batch_chunks, batch_urls, batch_embeddings):
                entities.append({"embedding": embedding.tolist(), "url": url})

        ids = collection.insert(entities)
        logger.info(f"Inserted {len(entities)} entities into Milvus")
        return ids
    except Exception as e:
        logger.error(f"Error inserting entities into Milvus: {str(e)}")
        return None

chunks = df['content'].tolist()  
urls = df['url'].tolist() 

inserted_ids = insert_chunks_into_milvus(chunks, urls, collection, model, batch_size=50)

if inserted_ids:
    print(f"Inserted IDs: {inserted_ids}")



INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
  0%|          | 0/132 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  1%|          | 1/132 [00:21<47:46, 21.88s/it]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

  2%|▏         | 2/132 [00:35<36:43, 16.95s/it]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

 99%|█████████▉| 131/132 [32:07<00:15, 15.27s/it]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 132/132 [32:18<00:00, 14.69s/it]
INFO:__main__:Inserted 6592 entities into Milvus


Inserted IDs: (insert count: 6592, delete count: 0, upsert count: 0, timestamp: 451199856348495875, success count: 6592, err count: 0, cost: 0)


In [14]:
#[CELL-10] Puhing the collection into the index

from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType, utility
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

if utility.has_collection(collection_name):
    
    collection = Collection(name=collection_name)

    try:
        collection.flush()
        logger.info(f"Data in collection '{collection_name}' flushed successfully.")
    except Exception as e:
        logger.error(f"Error flushing collection '{collection_name}': {str(e)}")

    index_params = {
        "index_type": "HNSW",
        "params": {"M": 16, "efConstruction": 200},
        "metric_type": "L2"
    }

    try:
        collection.create_index(field_name="embedding", index_params=index_params)
        logger.info(f"Index created successfully for collection '{collection_name}'.")
    except Exception as e:
        logger.error(f"Error creating index for collection '{collection_name}': {str(e)}")

    try:
        collection.load()
        logger.info(f"Collection '{collection_name}' loaded successfully.")
    except Exception as e:
        logger.error(f"Error loading collection '{collection_name}': {str(e)}")

else:
    logger.error(f"Collection '{collection_name}' does not exist.")



INFO:__main__:Connected to Milvus server.
INFO:__main__:Data in collection 'new_docs_v2' flushed successfully.
INFO:__main__:Index created successfully for collection 'new_docs_v2'.
INFO:__main__:Collection 'new_docs_v2' loaded successfully.


In [41]:
#[CELL-11] Checking if the data is inserted or not in the index.

num_entities = collection.num_entities
print(f"Number of entities in collection: {num_entities}")


Number of entities in collection: 6592


In [58]:
#[CELL-12] Retrieval and Re-ranking

from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline, DistilBertTokenizer, DistilBertForQuestionAnswering
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from pymilvus import Collection, connections, utility

collection_name = "new_docs_v2"
collection = Collection(name=collection_name)

index_params = {
    "index_type": "HNSW",
    "params": {"M": 16, "efConstruction": 200},
    "metric_type": "L2"
}
collection.create_index(field_name="embedding", index_params=index_params)

model = SentenceTransformer('all-MiniLM-L6-v2')

chunks = df['content'].tolist()
tokenized_chunks = [chunk.split() for chunk in chunks]
bm25 = BM25Okapi(tokenized_chunks)

def retrieve_and_rerank(query, k=3):
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_ranked_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:k]
    bm25_results = [chunks[i] for i in bm25_ranked_indices]

    query_embedding = model.encode(query)

    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}  
    results = collection.search([query_embedding.tolist()], "embedding", param=search_params, limit=k, output_fields=["url"])

    milvus_results = [result.entity.get("url") for result in results[0]]

    combined_results = list(set(bm25_results + milvus_results))


    combined_embeddings = [model.encode(text) for text in combined_results]
    similarity_scores = [float(query_embedding @ embedding.T) for embedding in combined_embeddings]
    reranked_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)
    reranked_results = [combined_results[i] for i in reranked_indices]

    return reranked_results




INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [59]:
#[CELL-13] Question Answering

qa_model_name = 'distilbert-base-cased-distilled-squad'
qa_pipeline = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased')

def retrieve_and_answer_questions(query, k=3):
    reranked_results = retrieve_and_rerank(query, k=k)

    answers = []
    for context in reranked_results:
        answer = qa_pipeline(question=query, context=context)
        answers.append(answer['answer'])

    return answers


In [60]:
# #[CELL-14] Sample Question Answering
query = "how to install cuda?"
answers = retrieve_and_answer_questions(query)
print("Answers to '{}' based on retrieved documents:".format(query))
for i, answer in enumerate(answers):
    print("{}. {}".format(i+1, answer))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]